In [ ]:
import allel
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from scipy.spatial.distance import cdist

In [ ]:
# # Perform PCA, 2, 4, 8, 20 Components
# pca_2= PCA(n_components=2)
# principal_components_train_2 = pca_2.fit_transform(X_train_std)

# pca_8 = PCA(n_components=8)
# principal_components_train_8 = pca_8.fit_transform(X_train_std)

# pca_20 = PCA(n_components=20)
# principal_components_train_20 = pca_20.fit_transform(X_train_std)

# pca_40 = PCA(n_components=40)
# principal_components_train_40 = pca_40.fit_transform(X_train_std)

# # Determine optimal number of clusters (k)
# num_clusters = 6

# # Apply K-Means clustering
# kmeans = KMeans(n_clusters=num_clusters, random_state=42)
# clusters_train_2 = kmeans.fit_predict(principal_components_train_2)
# clusters_train_8 = kmeans.fit_predict(principal_components_train_8)
# clusters_train_20 = kmeans.fit_predict(principal_components_train_20)
# clusters_train_40 = kmeans.fit_predict(principal_components_train_40)

# plt.subplot(2, 2, 1)
# scatter = plt.scatter(principal_components_train_2[:, 0], principal_components_train_2[:, 1], c=clusters_train_2, cmap='viridis')
# plt.xlabel('PC1')
# plt.ylabel('PC2')
# plt.title('PCA Space Clusters (2 Components)')
# plt.colorbar(scatter, label='Cluster')

# plt.subplot(2, 2, 2)
# scatter = plt.scatter(principal_components_train_8[:, 0], principal_components_train_8[:, 1], c=clusters_train_8, cmap='viridis')
# plt.xlabel('PC1')
# plt.ylabel('PC2')
# plt.title('PCA Space Clusters (8 Components)')
# plt.colorbar(scatter, label='Cluster')

# plt.subplot(2, 2, 3)
# scatter = plt.scatter(principal_components_train_20[:, 0], principal_components_train_20[:, 1], c=clusters_train_20, cmap='viridis')
# plt.xlabel('PC1')
# plt.ylabel('PC2')
# plt.title('PCA Space Clusters (20 Components)')
# plt.colorbar(scatter, label='Cluster')

# plt.subplot(2, 2, 4)
# scatter = plt.scatter(principal_components_train_40[:, 0], principal_components_train_40[:, 1], c=clusters_train_40, cmap='viridis')
# plt.xlabel('PC1')
# plt.ylabel('PC2')
# plt.title('PCA Space Clusters (40 Components)')
# plt.colorbar(scatter, label='Cluster')

# plt.tight_layout()
# plt.show()


# Applying LDA on PCA Components
1. Performing LDA
2. Plotting LDA Components

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# Initialize LDA
lda = LDA(n_components=num_clusters - 1)

# Fit LDA using PCA components and cluster labels
lda_components_train = lda.fit_transform(principal_components_train, clusters_train)


import pandas as pd
import seaborn as sns

# Create DataFrame for plotting
lda_df = pd.DataFrame(lda_components_train, columns=[f'LD{i+1}' for i in range(lda_components_train.shape[1])])
lda_df['Cluster'] = clusters_train
lda_df['SampleID'] = samples_train

# Plot first two LDA components
plt.figure(figsize=(8, 6))
scatter = plt.scatter(lda_components_train[:, 0], lda_components_train[:, 1], c=clusters_train, cmap='viridis')
plt.xlabel('LD1')
plt.ylabel('LD2')
plt.title('LDA of PCA Components')
plt.colorbar(scatter, label='Cluster')
plt.show()


# Preparing Data for the NN
1. Use LDA Components as Features
2. Split Data into Training and Testing Sets
3. Convert Labels to Categorical Format

## Explanation
Features: Using LDA components leverages the maximized class separation.

Data Splitting: Ensures model is evaluated on unseen data.

One-Hot Encoding: Converts integer labels to categorical format for multi-class classification.

In [ ]:
# Standardize the test data using the scaler fitted on training data
X_test_std = scaler.transform(X_test_raw)

# Apply PCA transformation using the PCA fitted on training data
principal_components_test = pca.transform(X_test_std)

# Note: We do NOT perform clustering on test data
# Instead, we will use the neural network to predict cluster labels

# Apply LDA transformation using the LDA fitted on training data
lda_components_test = lda.transform(principal_components_test)

In [ ]:
# Step 6: Prepare Data for Neural Network
# Features: LDA components
X_train = lda_components_train
X_test = lda_components_test

# Targets: Cluster labels for training data
y_train = clusters_train

# Since we don't have cluster labels for test data, we will predict them using the neural network
# For evaluation purposes, we can use the known clusters if we had them, but in this case, we don't

from tensorflow.keras.utils import to_categorical
# Convert training labels to categorical format
num_classes = num_clusters
y_train_cat = to_categorical(y_train, num_classes)

# Building and Training the Neural Network
1. Building the Neural Network
2. Compiling the Model
3. Training the Model

## Explanation
### Model Architecture:
- Input Layer: Size matches the number of LDA components.
- Hidden Layers: Include dense layers with ReLU activation.
- Output Layer: Uses softmax activation for multi-class classification.
### Compilation:
- Optimizer: Adam optimizer for efficient training.
- Loss Function: Categorical crossentropy for multi-class classification.
- Metrics: Accuracy to evaluate performance.
- Training Parameters:
- Epochs: Number of times the model sees the entire dataset.
- Validation Split: Fraction of training data used for validation.
- Batch Size: Number of samples processed before updating the model.

In [ ]:
from tensorflow.keras import layers, models, regularizers

# Define the model
model = models.Sequential()
model.add(layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(X_train, y_train_cat,
                    epochs=30,
                    validation_split=0.1,
                    batch_size=5,
                    callbacks=[early_stopping])

# Evaluating the Model
1. Evaluate on Test Set
2. Make Predictions
3. Classification Report
4. Confusion Matrix

## Explanation
Test Accuracy: Provides an overall performance metric on unseen data.

Classification Report: Includes precision, recall, f1-score for each class.

Confusion Matrix: Visualizes the model's performance in terms of correct and incorrect classifications for each class.

In [ ]:
# Step 8: Make Predictions on Test Data
# Predict cluster probabilities for test data
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

In [ ]:
# Step 9: Assign Cluster Labels to Test Data Based on Nearest Centroid
# Compute cluster centroids in LDA space for training data
lda_centroids = []
for i in range(num_clusters):
    cluster_data = lda_components_train[y_train == i]
    centroid = np.mean(cluster_data, axis=0)
    lda_centroids.append(centroid)
lda_centroids = np.array(lda_centroids)

# Assign cluster labels to test data based on nearest centroid in LDA space
distances = cdist(lda_components_test, lda_centroids)
y_test_assigned = np.argmin(distances, axis=1)

In [ ]:
# Step 10: Generate Confusion Matrix and Classification Report
cm = confusion_matrix(y_test_assigned, y_pred)

plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=range(num_classes), yticklabels=range(num_classes))
plt.xlabel('Predicted Cluster')
plt.ylabel('Assigned Cluster')
plt.title('Confusion Matrix')
plt.show()

print("Classification Report:")
print(classification_report(y_test_assigned, y_pred))